In [1]:
from tibet_data import Pairs, Pair
import pandas as pd

In [2]:
pairs = Pairs()
stdg = Pair(pairs, 'STDG')
stdg.get_volume()

1.630925192518

In [3]:
stdg.get_xch_reserve()

4.608706411428

In [4]:
stdg.get_annualized_yield()

0.06671858488354943

In [5]:
stdg.get_price_impact_threshold()

0.04608706411428

In [6]:
stdg.get_recovery_period()

350.3708042833882

In [7]:
usds = Pair(pairs, 'USDS')
usds.get_volume()

23.353681999121

In [8]:
usds.get_recovery_period()

24.468457327203442

In [9]:
incl4 = Pair(pairs, 'INCL4')
incl4.get_recovery_period()

254.21061255139045

In [10]:
incl4.get_recovery_period(0.1)

25.421061255139044

In [11]:
incl4.get_annualized_yield(), incl4.get_xch_reserve()

(0.08170736795352784, 5.222842128369)

In [12]:
incl4.get_volume()

2.247854901467

In [13]:
stdg.get_transactions(op='SWAP')

,pair_launcher_id,operation,height,coin_id,timestamp,state_change.xch,state_change.token,state_change.liquidity,time
0,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3791941,a18be0c4e4f78856809df9d68cd401d91b1986608de272...,1686526741,22554202054,-200000,0,2023-06-11 23:39:01
1,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3791821,64d775e5f56cbde3206fe61044784640c32374ed245b64...,1686524191,11194903423,-100000,0,2023-06-11 22:56:31
2,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783208,bd981a80d3968688bb0800373b0afbd88fc910f6f2494e...,1686365673,49999946955,-452990,0,2023-06-10 02:54:33
3,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783114,228ef267b62baefbcd69af238c25350b1904f9495112b5...,1686363923,19999976134,-184183,0,2023-06-10 02:25:23
4,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783017,02a71eb068f6076f435adf5d6c2ecfa788596ba0b1b4c9...,1686362585,499999986537,-5247188,0,2023-06-10 02:03:05
5,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3782598,20466e8d2cfecc7285ddc2b92bf3f35a374a942fafaf41...,1686354623,-466867310522,5000000,0,2023-06-09 23:50:23
6,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3775212,f409cd53c3f28acd260a5553ce0da2954c46e0b91adbc3...,1686216158,52612677973,-500000,0,2023-06-08 09:22:38
7,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3771037,da48e215f3b008bf5da4f30272da1a1dbcd0b660aa6002...,1686136123,200000000000,-2025017,0,2023-06-07 11:08:43
8,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3767064,3e9b740737c6a183dc54e2573e097dc3d210c072dfc4a2...,1686062718,-154143842754,1600000,0,2023-06-06 14:45:18
9,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3766871,4c6a176a3ba3c5a8296e182c11941b04928142a8d7c041...,1686059463,80000000000,-803282,0,2023-06-06 13:51:03


In [14]:
pairs.data.head()

,launcher_id,name,short_name,image_url,asset_id,current_coin_id,xch_reserve,token_reserve,liquidity,trade_volume
0,b7772fbcb3a297555f2dd75368ee85069c50d294902b40...,Stably USD,USDS,https://nftstorage.link/ipfs/bafybeibsolxdn3yn...,6d95dae356e32a71db5ddcb42224754a02524c615c5fc3...,ac62c299cbfd373a8e4a92cfdbe8b10b28d0d3ef3b155a...,343059898658450,11366853,11941763,360740594506141
1,636a5be72b9974d28393fbff1249e28dcdfb86b0bb9245...,Big Brain Token,BBT,https://bigbrain.at/bigbraintoken.png,9e04d998a90bca05bdbc301548c83104d79256f1b00bb2...,4ffa74283c1a7d41f5da0851470cf280809b7851700665...,182977000625777,211117622021,6112833045903,299897566346812
2,6dc7a439046ae61bf51e3af7e960ab691dbe6d58ddb289...,DeltaXtrade,DXT,https://nftstorage.link/ipfs/bafybeihklkvffcwg...,a7c8135b134e1a477563480febefd2e2091afe15d24ab5...,5b88ea0d81fe7c349232c5180d47aad35418e3c7b60a1d...,128235801668250,137577,158578,9373639859586
3,b62577aed2b09182f3db8b21049d687b597f1526fb8826...,PepeCoin,PEPE,https://nftstorage.link/ipfs/bafybeibtolqdipv4...,ec9d874e152e888231024c72e391fc484e8b6a1cf74443...,f9af69e66cae24bf6bc491f7dc0a5c9aefb20f8ee5263b...,123908743276887,49798973953,38790482670,375997086988279
4,c0952d9c3761302a5545f4c04615a855396b1570be5c61...,Dexie Bucks,DBX,https://nftstorage.link/ipfs/bafybeidiupubzjzn...,db1a9020d48d9d4ad22631b66ab4b9ebd3637ef7758ad3...,0f8a6145900e94d30457b1f87939cc1bfa49478e8f0e38...,108223385623664,37707141,31594406,118153167218781


In [15]:
pepe = Pair(pairs, 'PEPE')
pepe.get_recovery_period()

14.796402850991285

In [16]:
pepe.get_annualized_yield()

0.058538168850058137